In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

https://www.kaggle.com/code/faressayah/cifar-10-images-classification-using-cnns-88

In [ ]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test)  = tf.keras.datasets.cifar10.load_data()

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Define the labels of the dataset
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck']

# 서브플롯(그래프)의 크기를 정의 가로10개, 세로 10개의 총 100개의 그래프가 있도록 10x10 크기의 서브플롯
W_grid = 10
L_grid = 10

# fig, axes = plt.subplots(L_grid, W_grid)
# subplot return the figure object and axes object
# 'axes' 객체를 사용하여 여러 위치에 대한 그림

fig, axes = plt.subplots(L_grid, W_grid, figsize = (17,17))

axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_train = len(X_train) # get the length of the train dataset

# Select a random number from 0 to n_train
for i in np.arange(0, W_grid * L_grid): # create evenly spaces variables 

    # Select a random number
    index = np.random.randint(0, n_train)
    # read and display an image with the selected index    
    axes[i].imshow(X_train[index,1:])
    label_index = int(y_train[index])
    axes[i].set_title(labels[label_index], fontsize = 8)
    axes[i].axis('off')

plt.subplots_adjust(hspace=0.4)

In [ ]:
classes_name = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

classes, counts = np.unique(y_train, return_counts=True)
plt.barh(classes_name, counts)
plt.title('Class distribution in training set')

In [ ]:
classes, counts = np.unique(y_test, return_counts=True)
plt.barh(classes_name, counts)
plt.title('Class distribution in testing set')

# 스케일링

y_train 변수가 [0, 1, 2, 3, ..., 8, 9]와 같은 정수값들로 이루어져 있다면, y_cat_train 변수는 [1 0 0 0 0 0 0 0 0 0], [0 1 0 0 0 0 0 0 0 0], [0 0 1 0 0 0 0 0 0 0], [0 0 0 1 0 0 0 0 0 0], ..., [0 0 0 0 0 0 0 0 0 1]과 같이, 각 레이블을 크기가 10인 벡터로 변환한 결과가 저장

이러한 원-핫 인코딩은 다중 클래스 분류 문제에서 자주 사용 분류 모델의 출력층에서 각 클래스에 대한 확률을 계산할 때, 각 클래스에 대한 원-핫 인코딩된 벡터를 사용하여 계산

In [ ]:
# 스케일링
X_train = X_train / 255.0
X_test = X_test / 255.0

# to_categorical() 함수 : y_train 변수를 one-hot encoding하여 y_cat_train 변수에 저장하는 코드
# to_categorical() 함수는 입력된 정수형 레이블을 원-핫 인코딩된 벡터로 변환
# 첫 번째 인자로는 변환할 정수형 레이블이 들어가고, 두 번째 인자로는 원-핫 인코딩된 벡터의 크기를 지정
#  y_train 변수의 값들이 0부터 9까지의 정수값을 가지기 때문에, 두 번째 인자로 10을 전달하여 각 레이블을 크기가 10인 원-핫 인코딩된 벡터로 변환

y_cat_train = to_categorical(y_train, 10)
y_cat_test = to_categorical(y_test, 10)

In [ ]:
y_cat_train

# 모델링 

## Keras사용 구현 CNN모델 정의,컴파일

In [ ]:
# 입력 이미지 : 32x32 픽셀, RGB채널
INPUT_SHAPE = (32, 32, 3)
# 컨볼루션 레이어에서 사용할 필터의 
KERNEL_SIZE = (3, 3)
# sequential함수로 빈 모델 
# 모델에 + 컨불루션 레이어 (레이어마다 활성화 함수 : ReLU , BatchNormalization(데이터 정규화,학습 속도 높여서 모델의 성능을 개선))
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization()) 

# Pooling layer : MaxPooling레이어 사용 -> 다운 샘플링
model.add(MaxPool2D(pool_size=(2, 2)))

# Dropout layers : 오버피팅 방지
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 평탄화
model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu')) # Fully-Connected 레이어를 추가하여 분류 문제를 푸는데 사용
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax')) # 마지막 레이어 : Softmax 함수를 사용하여 10개의 클래스 중 하나에 속하는 확률을 출력.

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
# 모델 컴파일
# 모델의 학습 과정에서 사용될 손실 함수(loss)와 옵티마이저(optimizer)를 지정
# 분류문제 : categorical_crossentropy
# 옵티마이저 : Adam
# 평가 지표 : accuracy, precision, recall
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

In [ ]:
model.summary()

### Eary Stopping

* Keras에서 제공하는 콜백(Callback) 함수 중 하나인 EarlyStopping
* 모델 학습 조기 종료
* monitor 인자로 모니터링할 지표를 지정 ,이 경우에는 검증 데이터셋의 손실(loss)을 모니터링
* patience 인자로 지정된 에포크 만큼의 지속적인 개선이 없으면 학습을 종료
* 2로 설정되어 있기 때문에, 검증 데이터셋의 손실이 2번 연속으로 개선되지 않으면 학습을 조기 종료

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

### 데이터 fit

* ImageDataGenerator를 사용하여 이미지 데이터 증강(augmentation)을 수행, 생성된 이미지 데이터를 이용해 모델을 학습하는 방법

* 인자 : 다양한 이미지 변형 관련 파라미터들이 전달
    * width_shift_range : 이미지를 좌우 이동시키는 방법으로 데이터를 증강
    * height_shift_range : 이미지 상하 이동시키는 방법
    * horizontal_flip을 이용하여 좌우 반전을 추가적으로 적용
    
* data_generator.flow() 메소드: 생성된 이미지 데이터와 one-hot 인코딩된 레이블 데이터를 배치(batch) 단위로 가져오며, batch_size를 인자로 전달하여 한 번에 가져올 배치 크기를 지정

* bath_size : 모델이 한 번에 처리하는 데이터 샘플의 수를 결정하는 파라미터입니다. 즉, 데이터를 몇 개씩 묶어서 모델에 입력할 것인지를 지정하는 값

* batch_size 값이 작을수록 메모리 사용량은 적어지지만, 처리 속도는 느려짐, 더 많은 업데이트가 가능하므로 모델이 더 빨리 수렴할 가능성이 높아짐
* batch_size 값이 클수록 메모리 사용량은 많아지지만, 처리 속도는 빨라짐  더 안정적인 학습이 가능

In [ ]:
batch_size = 32
data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(X_train, y_cat_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

r = model.fit(train_generator, 
              epochs=50,
              steps_per_epoch=steps_per_epoch,
              validation_data=(X_test, y_cat_test), 
#               callbacks=[early_stop],
#               batch_size=batch_size, # 한번에 32개의 데이터 샘플을 처리
             )

# 모델평가

In [ ]:
plt.figure(figsize=(12, 16))

# history : 학습이력
plt.subplot(4, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='val_Loss')
plt.title('Loss Function Evolution')
plt.legend()

plt.subplot(4, 2, 2)
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.title('Accuracy Function Evolution')
plt.legend()

plt.subplot(4, 2, 3)
plt.plot(r.history['precision'], label='precision')
plt.plot(r.history['val_precision'], label='val_precision')
plt.title('Precision Function Evolution')
plt.legend()

plt.subplot(4, 2, 4)
plt.plot(r.history['recall'], label='recall')
plt.plot(r.history['val_recall'], label='val_recall')
plt.title('Recall Function Evolution')
plt.legend()

학습된 모델을 사용하여 테스트 데이터셋에 대한 정확도를 계산하고, confusion matrix를 시각화하여 모델의 분류 성능을 평가

In [ ]:
evaluation = model.evaluate(X_test, y_cat_test)
print(f'Test Accuracy : {evaluation[1] * 100:.2f}%')

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test, y_pred)


disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=labels)


# NOTE: Fill all variables here with default values of the plot_confusion_matrix
fig, ax = plt.subplots(figsize=(10, 10))
disp = disp.plot(xticks_rotation='vertical', ax=ax,cmap='summer')

plt.show()

 confusion matrix에서 대각선의 값(정확하게 분류한 데이터 개수)이 높은 것으로 보아, 각 클래스마다 꽤 균형있게 분류하는 것을 확인

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from tensorflow.keras.models import load_model

model.save('cnn_20_epochs.h5')

# Test

In [ ]:
pip install torch

In [ ]:
pip install torchvision

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
model = torch.load('jibeen_model.pt')
image = Image.open('test_image.jpg')
image = transform(image).unsqueeze(0)
outputs = model(image)
_, predicted = torch.max(outputs, 1)

# 결과 출력
print('Predicted:', predicted.item())

# 이미지 시각화
image = image.squeeze(0)
image = transforms.ToPILImage()(image)
plt.imshow(image)
plt.show()

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
model = tf.keras.models.load_model('./cnn_20_epochs.h5')
img = tf.keras.preprocessing.image.load_img('C:/Users/Playdata/Desktop/배2.jpg', target_size=(32, 32))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
img_array /= 255.
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
class_index = np.argmax(score)
class_name = class_names[class_index]
print("This image most likely belongs to {} with a {:.2f} percent confidence."
      .format(class_name, 100 * np.max(score)))
plt.imshow(img)

In [ ]:
my_image = X_test[100]
plt.imshow(my_image)

# 사슴으로 확인
print(f" Image 100 is {y_test[100]}")

# correctly predicted as a Deer
pred_100 = np.argmax(model.predict(my_image.reshape(1, 32, 32, 3)))
print(f"The model predict that image 100 is {pred_100}")

In [ ]:
# Define the labels of the dataset
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck']

# Let's view more images in a grid format
# Define the dimensions of the plot grid 
W_grid = 10
L_grid = 10

# fig, axes = plt.subplots(L_grid, W_grid)
# subplot return the figure object and axes object
# we can use the axes object to plot specific figures at various locations

fig, axes = plt.subplots(L_grid, W_grid, figsize = (17,17))

axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_test = len(X_test) # get the length of the train dataset

# Select a random number from 0 to n_train
for i in np.arange(0, W_grid * L_grid): # create evenly spaces variables 

    # Select a random number
    index = np.random.randint(0, n_test)
    # read and display an image with the selected index    
    axes[i].imshow(X_test[index,1:])
    label_index = int(y_pred[index])
    axes[i].set_title(labels[label_index], fontsize = 8)
    axes[i].axis('off')

plt.subplots_adjust(hspace=0.4)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel(f"{labels[int(predicted_label)]} {100*np.max(predictions_array):2.0f}% ({labels[int(true_label)]})", 
               color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, int(true_label[i])
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
predictions = model.predict(X_test)

# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 8
num_cols = 5
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], y_test, X_test)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], y_test)
plt.tight_layout()
plt.show()

# 이미지분류를 위한 DenseNet model 

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()
base_model = DenseNet121(input_shape=(32, 32, 3), include_top=False, weights='imagenet', pooling='avg')
model.add(base_model)
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

r = model.fit(train_generator, 
              epochs=100,
              steps_per_epoch=steps_per_epoch,
              validation_data=(X_test, y_cat_test), 
#               callbacks=[early_stop],
             )

# 모델 저장

In [2]:
from tensorflow.keras.models import load_model

# 모델 사이트

In [3]:
import requests
import base64
url = "http://127.0.0.1:8000/cnnpredict/"
with open(r"C:\Users\Playdata\Desktop\숫사슴.jpg", 'rb') as img:
    base64_string = base64.b64encode(img.read())
r = requests.post(url, data = {'image' : base64_string})
r.text

'{"result":"deer"}'